#### 한-영번역기 만들기

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import collections

import helper
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11954919717749950129
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14674281152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12878680984660920125
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [5]:
import os, sys, time
import pandas as pd
import numpy as np

df1 = pd.read_excel('/content/drive/MyDrive/machine_translator_project/data/1_구어체(1)_200226 (1).xlsx')
df2 = pd.read_excel('/content/drive/MyDrive/machine_translator_project/data/1_구어체(1)_200226.xlsx')

df = pd.concat([df1,df2])
print(df.info()) # check rows
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   SID     400000 non-null  int64 
 1   원문      400000 non-null  object
 2   번역문     400000 non-null  object
dtypes: int64(1), object(2)
memory usage: 12.2+ MB
None


,SID,원문,번역문
0,1,'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...,Bible Coloring' is a coloring application that...
1,2,씨티은행에서 일하세요?,Do you work at a City bank?
2,3,푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.,"PURITO's bestseller, which recorded 4th rough ..."
3,4,11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.,In Chapter 11 Jesus called Lazarus from the to...
4,5,"6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.",I would feel grateful to know how many stocks ...


In [12]:
df.reset_index(inplace=True)

In [13]:
df_korean_sentences = df.loc[1:50000, '원문']
kor = df_korean_sentences.tolist()
df_english_sentences = df.loc[1:50000, '번역문']
eng = df_english_sentences.tolist()
print(len(kor))
print(len(eng)) # 50,000 rows

50000
50000


In [14]:
# !pip install konlpy
from konlpy.tag import Okt 

In [15]:
def sen2morph(sentence):    
    morphs = []    
    analyzer = Okt()
    morphs = analyzer.morphs(sentence)
    return morphs
morphs = sen2morph(kor[0])
morphs

['씨티', '은행', '에서', '일', '하세요', '?']

In [16]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []
    
    for korean_sentence in kor:
            src_line_input = sen2morph(korean_sentence)
            encoder_input.append(src_line_input)
        
    for english_sentence in eng:
            tar_line_input = [w for w in ("<sos> " + english_sentence).split()]
            tar_line_target = [w for w in (english_sentence + " <eos>").split()]
            decoder_input.append(tar_line_input)
            decoder_target.append(tar_line_target)

    return encoder_input, decoder_input, decoder_target

In [17]:
sents_ko_in, sents_en_in, sents_en_out = load_preprocessed_data()
print(sents_ko_in[:2])
print(sents_en_in[:2])
print(sents_en_out[:2])

[['씨티', '은행', '에서', '일', '하세요', '?'], ['푸리', '토의', '베스트셀러', '는', '해외', '에서', '입', '소문', '만으로', '4', '차', '완판', '을', '기록', '하였다', '.']]
[['<sos>', 'Do', 'you', 'work', 'at', 'a', 'City', 'bank?'], ['<sos>', "PURITO's", 'bestseller,', 'which', 'recorded', '4th', 'rough', '-cuts', 'by', 'words', 'of', 'mouth', 'from', 'abroad.']]
[['Do', 'you', 'work', 'at', 'a', 'City', 'bank?', '<eos>'], ["PURITO's", 'bestseller,', 'which', 'recorded', '4th', 'rough', '-cuts', 'by', 'words', 'of', 'mouth', 'from', 'abroad.', '<eos>']]


#### tokenizing

In [18]:
tokenizer_ko = Tokenizer(filters="", lower=False)
tokenizer_ko.fit_on_texts(sents_ko_in)
encoder_input = tokenizer_ko.texts_to_sequences(sents_ko_in)

In [19]:
encoder_input = pad_sequences(encoder_input, padding="post")

In [20]:
tokenizer_en = Tokenizer(filters="", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
tokenizer_en.fit_on_texts(sents_en_out)
decoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
decoder_target = tokenizer_en.texts_to_sequences(sents_en_out)

In [21]:
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

In [22]:
src_vocab_size = len(tokenizer_ko.word_index) + 1
print("한국어 단어 집합의 크기 : {:d}".format(src_vocab_size))

tar_vocab_size = len(tokenizer_en.word_index) + 1
print("영어 단어 집합의 크기 : {:d}".format(tar_vocab_size))

src_to_index = tokenizer_ko.word_index
index_to_src = tokenizer_ko.index_word # 훈련 후 결과 비교할 때 사용
tar_to_index = tokenizer_en.word_index # 훈련 후 예측 과정에서 사용
index_to_tar = tokenizer_en.index_word # 훈련 후 결과 비교할 때 사용

한국어 단어 집합의 크기 : 38546
영어 단어 집합의 크기 : 39741


In [23]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[ 2912 32716  9408 ... 20967 34480  7803]


In [24]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [26]:
n_of_val = int(50000*0.2)
print(n_of_val)

10000


In [27]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [28]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(40000, 37)
(40000, 40)
(40000, 40)
(10000, 37)
(10000, 40)
(10000, 40)


In [29]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

latent_dim = 512

In [30]:
# 인코더
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(src_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state=True) # 상태값 리턴을 위해 return_state는 True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉 상태와 셀 상태를 리턴
encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True, 모든 시점에 대해서 단어를 예측하기 위해 return_sequences는 True
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state=encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 512)    19735552    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 512)    20347392    input_2[0][0]                    
______________________________________________________________________________________________

In [31]:
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 128, epochs = 10)

Epoch 1/10
313/313 [==============================] - 192s 483ms/step - loss: 2.5913 - acc: 0.7137 - val_loss: 1.7422 - val_acc: 0.7607
Epoch 2/10
313/313 [==============================] - 150s 479ms/step - loss: 1.6850 - acc: 0.7638 - val_loss: 1.6264 - val_acc: 0.7708
Epoch 3/10
313/313 [==============================] - 152s 486ms/step - loss: 1.5722 - acc: 0.7746 - val_loss: 1.5731 - val_acc: 0.7779
Epoch 4/10
313/313 [==============================] - 152s 486ms/step - loss: 1.4862 - acc: 0.7828 - val_loss: 1.5304 - val_acc: 0.7817
Epoch 5/10
313/313 [==============================] - 152s 486ms/step - loss: 1.4238 - acc: 0.7878 - val_loss: 1.5055 - val_acc: 0.7839
Epoch 6/10
313/313 [==============================] - 152s 487ms/step - loss: 1.3590 - acc: 0.7936 - val_loss: 1.4892 - val_acc: 0.7860
Epoch 7/10
313/313 [==============================] - 152s 486ms/step - loss: 1.3006 - acc: 0.7993 - val_loss: 1.4758 - val_acc: 0.7876
Epoch 8/10
313/313 [============================

In [32]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2= dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)



In [33]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

    # stop_condition이 True가 될 때까지 루프 반복
    # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

         # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        '''
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > 50):
            stop_condition = True
        '''
        if sampled_char == '<eos>':
            stop_condition = True            

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [34]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + index_to_src[i]+' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_to_index['<sos>']) and i!=tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp
    

In [35]:
for seq_index in [3,50,100,300,1001]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print("원문 : ",seq2src(encoder_input_train[seq_index]))
    print("번역문 :",seq2tar(decoder_input_train[seq_index]))
    print("예측문 :",decoded_sentence[:-5])
    print("\n")

원문 :  그렇다고 너무 낙심 하지는 마세요 . 
번역문 : Don't be so disappointed. 
예측문 :  I am not good at all. 


원문 :  나 는 늦게까지 교수 님 의 작업 을 도 왔어요 . 
번역문 : I helped the professor's work till late. 
예측문 :  I have a lot of time to go to the school. 


원문 :  그럼 네 가 말 한 건 , 한국 에서 남은 학기 등록금 과 생활비 를 내줄 수 있다는 건가 요 ? 
번역문 : Then what you said, did you mean that you can support the education fee and living expenses in Korea for the left semester? 
예측문 :  Then, is it okay to you and the Korean friends will be able to see the product with the same as soon as possible. 


원문 :  8월 12일 에 1 에서 2 로 , 7 에서 8 로 교체 하였고 . 
번역문 : It has been altered from 1 to 2, 7 to 8 on August 12th. 
예측문 :  I am sending the second floor to the 1st floor of the second floor of the 2nd floor. 


원문 :  나 는 남자 친구 를 멀리 서 봐야 했어요 . 
번역문 : I had to see my boyfriend from far away. 
예측문 :  I went to the same as I was in a high school. 


